<a href="https://colab.research.google.com/github/angelaaaateng/AI_Projects/blob/main/Day5/transform_tabular_data_to_pdf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
!pip install openai==0.28.1
!pip install fpdf==1.7.2

transform pdf to text and then Q&A dataset (like for finetuning)

In [58]:
import pandas as pd
import openai
import time
from fpdf import FPDF


In [59]:
df = pd.read_csv('https://raw.githubusercontent.com/ALGOREX-PH/Day-4-AI-First-Dataset-Live/refs/heads/main/Parcel_Information_Dataset.csv')

In [60]:
df.sample(5)

,Parcel ID,Sender Name,Sender Address,Sender Contact,Receiver Name,Receiver Address,Receiver Contact,Parcel Weight (kg),Parcel Dimensions (cm),Shipment Date,...,Shipping Cost ($),Shipping Method,Tracking Number,Insurance Value ($),Fragile,Temperature Sensitive,Special Instructions,Delivery Attempt Count,Last Location,Courier ID
19,PCL437818,Chris White,"101 Pine St, Houston, TX 77001",+1-555-3227,Nancy Wilson,"101 Pine St, Houston, TX 77001",+1-555-8283,12.76,46x87x47,2024-11-29,...,38.23,Standard,TRK734214037,419.78,Yes,No,Deliver to reception,3,"Distribution Center, Phoenix, AZ",CR6547
12,PCL282859,Alice Green,"101 Pine St, Houston, TX 77001",+1-555-6289,Nancy Wilson,"789 Oak Rd, Chicago, IL 60601",+1-555-7344,12.89,74x66x41,2024-11-15,...,8.91,Express,TRK927844751,479.58,No,No,Ring bell,3,"Distribution Center, Los Angeles, CA",CR9025
2,PCL104639,John Doe,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-3768,Bob Johnson,"123 Elm St, New York, NY 10001",+1-555-9256,5.65,45x39x53,2024-11-09,...,39.66,Express,TRK696430980,325.55,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR8897
1,PCL761243,John Doe,"101 Pine St, Houston, TX 77001",+1-555-2574,Sara Davis,"123 Elm St, New York, NY 10001",+1-555-3140,11.91,43x64x11,2024-11-23,...,5.78,Standard,TRK278841245,344.57,Yes,No,NaN,1,"Distribution Center, Phoenix, AZ",CR8429
16,PCL795720,Chris White,"123 Elm St, New York, NY 10001",+1-555-2103,Nancy Wilson,"101 Pine St, Houston, TX 77001",+1-555-9613,7.78,81x46x85,2024-11-06,...,34.62,Standard,TRK254993956,177.02,Yes,Yes,Ring bell,1,"Distribution Center, Los Angeles, CA",CR7080


In [61]:
df.shape

(20, 22)

In [62]:
df.head()

,Parcel ID,Sender Name,Sender Address,Sender Contact,Receiver Name,Receiver Address,Receiver Contact,Parcel Weight (kg),Parcel Dimensions (cm),Shipment Date,...,Shipping Cost ($),Shipping Method,Tracking Number,Insurance Value ($),Fragile,Temperature Sensitive,Special Instructions,Delivery Attempt Count,Last Location,Courier ID
0,PCL398114,Michael Brown,"456 Maple Ave, Los Angeles, CA 90001",+1-555-4253,Sara Davis,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-8367,7.75,97x16x70,2024-11-17,...,8.66,Standard,TRK389255020,257.34,Yes,No,NaN,1,"Distribution Center, Phoenix, AZ",CR9571
1,PCL761243,John Doe,"101 Pine St, Houston, TX 77001",+1-555-2574,Sara Davis,"123 Elm St, New York, NY 10001",+1-555-3140,11.91,43x64x11,2024-11-23,...,5.78,Standard,TRK278841245,344.57,Yes,No,NaN,1,"Distribution Center, Phoenix, AZ",CR8429
2,PCL104639,John Doe,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-3768,Bob Johnson,"123 Elm St, New York, NY 10001",+1-555-9256,5.65,45x39x53,2024-11-09,...,39.66,Express,TRK696430980,325.55,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR8897
3,PCL768961,Linda Black,"456 Maple Ave, Los Angeles, CA 90001",+1-555-2027,Sara Davis,"202 Birch Blvd, Phoenix, AZ 85001",+1-555-3973,15.05,23x99x83,2024-11-18,...,45.52,Express,TRK588614196,218.93,No,No,Handle with care,3,"Distribution Center, Houston, TX",CR4388
4,PCL935942,Alice Green,"101 Pine St, Houston, TX 77001",+1-555-3594,Mike Lee,"101 Pine St, Houston, TX 77001",+1-555-8828,13.20,43x83x93,2024-11-18,...,9.07,Express,TRK897242703,255.49,No,No,Deliver to reception,1,"Distribution Center, Phoenix, AZ",CR9727


In [63]:
# Get Column Names and Data Types
column_info = df.dtypes
print("Column Names and Data Types:\n", column_info, "\n")

# Identify Categorical and Numerical Features
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
numerical_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()

print("Categorical Columns:", categorical_cols)
print("Numerical Columns:", numerical_cols, "\n")

Column Names and Data Types:
 Parcel ID                   object
Sender Name                 object
Sender Address              object
Sender Contact              object
Receiver Name               object
Receiver Address            object
Receiver Contact            object
Parcel Weight (kg)         float64
Parcel Dimensions (cm)      object
Shipment Date               object
Estimated Delivery Date     object
Delivery Status             object
Shipping Cost ($)          float64
Shipping Method             object
Tracking Number             object
Insurance Value ($)        float64
Fragile                     object
Temperature Sensitive       object
Special Instructions        object
Delivery Attempt Count       int64
Last Location               object
Courier ID                  object
dtype: object 

Categorical Columns: ['Parcel ID', 'Sender Name', 'Sender Address', 'Sender Contact', 'Receiver Name', 'Receiver Address', 'Receiver Contact', 'Parcel Dimensions (cm)', 'Shipment Date'

In [64]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20 entries, 0 to 19
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   Parcel ID                20 non-null     object 
 1   Sender Name              20 non-null     object 
 2   Sender Address           20 non-null     object 
 3   Sender Contact           20 non-null     object 
 4   Receiver Name            20 non-null     object 
 5   Receiver Address         20 non-null     object 
 6   Receiver Contact         20 non-null     object 
 7   Parcel Weight (kg)       20 non-null     float64
 8   Parcel Dimensions (cm)   20 non-null     object 
 9   Shipment Date            20 non-null     object 
 10  Estimated Delivery Date  20 non-null     object 
 11  Delivery Status          20 non-null     object 
 12  Shipping Cost ($)        20 non-null     float64
 13  Shipping Method          20 non-null     object 
 14  Tracking Number          20 

In [65]:
df.dtypes

,0
Parcel ID,object
Sender Name,object
Sender Address,object
Sender Contact,object
Receiver Name,object
Receiver Address,object
Receiver Contact,object
Parcel Weight (kg),float64
Parcel Dimensions (cm),object
Shipment Date,object


In [66]:
# Get Unique Values for Categorical Features with Limitation
def get_unique_values(col, max_display=10):
    uniques = df[col].unique().tolist()
    if len(uniques) > max_display:
        return uniques[:max_display] + ["..."]
    return uniques

unique_values = {col: get_unique_values(col) for col in categorical_cols}

print("Unique Values for Categorical Features:")
for col, uniques in unique_values.items():
    print(f" - {col}: {uniques}")
print()

Unique Values for Categorical Features:
 - Parcel ID: ['PCL398114', 'PCL761243', 'PCL104639', 'PCL768961', 'PCL935942', 'PCL595560', 'PCL409380', 'PCL616284', 'PCL137312', 'PCL418086', '...']
 - Sender Name: ['Michael Brown', 'John Doe', 'Linda Black', 'Alice Green', 'Chris White']
 - Sender Address: ['456 Maple Ave, Los Angeles, CA 90001', '101 Pine St, Houston, TX 77001', '202 Birch Blvd, Phoenix, AZ 85001', '123 Elm St, New York, NY 10001', '789 Oak Rd, Chicago, IL 60601']
 - Sender Contact: ['+1-555-4253', '+1-555-2574', '+1-555-3768', '+1-555-2027', '+1-555-3594', '+1-555-1235', '+1-555-7417', '+1-555-7648', '+1-555-5674', '+1-555-8484', '...']
 - Receiver Name: ['Sara Davis', 'Bob Johnson', 'Mike Lee', 'Jane Smith', 'Nancy Wilson']
 - Receiver Address: ['202 Birch Blvd, Phoenix, AZ 85001', '123 Elm St, New York, NY 10001', '101 Pine St, Houston, TX 77001', '789 Oak Rd, Chicago, IL 60601']
 - Receiver Contact: ['+1-555-8367', '+1-555-3140', '+1-555-9256', '+1-555-3973', '+1-555-88

In [67]:
df.nunique()

,0
Parcel ID,20
Sender Name,5
Sender Address,5
Sender Contact,20
Receiver Name,5
Receiver Address,4
Receiver Contact,20
Parcel Weight (kg),20
Parcel Dimensions (cm),20
Shipment Date,16


In [68]:
# for column in df.columns:
#     unique_values = df[column].unique()
#     print(f"Unique values in '{column}' column: {unique_values}")

In [69]:
# Get Descriptive Statistics for Numerical Features
numerical_stats = df[numerical_cols].describe()
print("Descriptive Statistics of Numerical Features:\n", numerical_stats, "\n")

Descriptive Statistics of Numerical Features:
        Parcel Weight (kg)  Shipping Cost ($)  Insurance Value ($)  \
count             20.0000          20.000000            20.000000   
mean               9.7480          26.450500           292.237500   
std                5.3491          13.566969           102.527009   
min                1.6400           5.780000            63.900000   
25%                5.2700          13.760000           223.992500   
50%                9.5150          27.400000           277.725000   
75%               13.2350          38.207500           374.115000   
max               18.1200          45.520000           479.580000   

       Delivery Attempt Count  
count               20.000000  
mean                 1.900000  
std                  0.852242  
min                  1.000000  
25%                  1.000000  
50%                  2.000000  
75%                  3.000000  
max                  3.000000   



In [70]:
numerical_stats

,Parcel Weight (kg),Shipping Cost ($),Insurance Value ($),Delivery Attempt Count
count,20.0000,20.000000,20.000000,20.000000
mean,9.7480,26.450500,292.237500,1.900000
std,5.3491,13.566969,102.527009,0.852242
min,1.6400,5.780000,63.900000,1.000000
25%,5.2700,13.760000,223.992500,1.000000
50%,9.5150,27.400000,277.725000,2.000000
75%,13.2350,38.207500,374.115000,3.000000
max,18.1200,45.520000,479.580000,3.000000


In [71]:
def generate_template(df, column_info, categorical_cols, numerical_cols, unique_values, numerical_stats):

    # Construct a summary of the dataframe's structure
    column_summary = "Column Names and Data Types:\n"
    for col, dtype in column_info.items():
        column_summary += f"- {col} ({dtype})\n"

    # Unique values for categorical features
    unique_values_str = "Unique Values for Categorical Features:\n"
    for col, uniques in unique_values.items():
        unique_values_str += f"- {col}: {uniques}\n"

    # Descriptive statistics for numerical features
    numerical_stats_str = "Descriptive Statistics of Numerical Features:\n"
    for col in numerical_cols:
        numerical_stats_str += f"- {col}:\n"
        for stat_name, value in numerical_stats[col].items():
            numerical_stats_str += f"    {stat_name}: {value}\n"

    # Define the system prompt
    system_prompt = """You are an intelligent assistant that creates descriptive templates for transforming dataframe rows into coherent paragraphs.
    Analyze the provided dataframe structure and generate a template sentence that includes placeholders for each column.
    Ensure the template is contextually relevant and maintains grammatical correctness."""

    # Define the user prompt
    user_prompt = f"""
    Analyze the following dataframe structure and create a descriptive template with placeholders for each column.

    <column_summary>
    {column_summary}
    </column_summary>

    <unique_values>
    {unique_values_str}
    </unique_values>

    <numerical_stats>
    {numerical_stats_str}
    </numerical_stats>

    Use the exact column names from the column_summary in generating the variable names in the template,
    as they will be populated with the actual values in the dataset.

    Example Template about a Spotify dataset:
    "{{artist}} gained {{streams}} streams in the song '{{song}}' that was a hit in {{date}}."

    - Output only the template without any explanation or introduction.
    - The template's variables will be dynamically replaced so make sure they're formatted properly.
    """

    # Generate the template (with retries)
    retries = 3
    for attempt in range(retries):
        try:
            response = openai.ChatCompletion.create(
                model="gpt-4o-mini",
                temperature=0.3,
                max_tokens=1024,
                messages=[
                    {"role": "system", "content": system_prompt},
                    {"role": "user", "content": user_prompt}
                ]
            )
            template = response['choices'][0]['message']['content'].strip()
            return template
        except Exception as e:
            print(f"Error generating template (Attempt {attempt + 1}/{retries}): {e}")
            time.sleep(2)  # Wait before retrying

    return None

In [72]:
template = generate_template(df, column_info, categorical_cols, numerical_cols, unique_values, numerical_stats)
template

'"Parcel ID {Parcel ID}, sent by {Sender Name} from {Sender Address} (Contact: {Sender Contact}), is addressed to {Receiver Name} at {Receiver Address} (Contact: {Receiver Contact}). The parcel weighs {Parcel Weight (kg)} kg and has dimensions of {Parcel Dimensions (cm)}. It was shipped on {Shipment Date} with an estimated delivery date of {Estimated Delivery Date}. The current delivery status is \'{Delivery Status}\', and it was shipped using the {Shipping Method} method at a cost of ${Shipping Cost ($)}. The tracking number for this shipment is {Tracking Number}, and it is insured for ${Insurance Value ($)}. The parcel is marked as {Fragile} and {Temperature Sensitive}. Special instructions include: \'{Special Instructions}\'. There have been {Delivery Attempt Count} delivery attempts, and the last known location of the parcel was {Last Location}. The courier responsible for this delivery is identified as {Courier ID}."'

In [73]:
def save_content_to_txt(paragraphs, txt_filename):
    try:
        with open(txt_filename, 'w', encoding='utf-8') as file:

            # Write each paragraph
            for para in paragraphs:
                file.write(para + '\n\n')  # Double newline for separation

        print(f"All content has been saved as '{txt_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to text file: {e}")

In [74]:
def save_content_to_pdf(paragraphs, pdf_filename):
    try:
        pdf = FPDF()
        pdf.add_page()
        pdf.set_auto_page_break(auto=True, margin=15)
        pdf.set_font("Arial", size=12)

        # Add each paragraph
        for para in paragraphs:
            pdf.multi_cell(0, 10, para)
            pdf.ln()  # Add a line break between paragraphs

        pdf.output(pdf_filename)
        print(f"All content has been saved as '{pdf_filename}'.\n")
    except Exception as e:
        print(f"Error saving content to PDF: {e}")

In [75]:
# Generate the template
template = generate_template(df, column_info, categorical_cols, numerical_cols, unique_values, numerical_stats)

if template:
    print("\nGenerated Template:\n", template, "\n")
else:
    print("Failed to generate template.\n")



Generated Template:
 "Parcel ID {Parcel ID} was shipped by {Sender Name} from {Sender Address} with the contact number {Sender Contact}. It is addressed to {Receiver Name} at {Receiver Address}, who can be reached at {Receiver Contact}. The parcel weighs {Parcel Weight (kg)} kg and has dimensions of {Parcel Dimensions (cm)}. The shipment was sent on {Shipment Date} and is estimated to be delivered by {Estimated Delivery Date}. Currently, the delivery status is '{Delivery Status}' and it was shipped via {Shipping Method} at a cost of ${Shipping Cost ($)}. The tracking number for this parcel is {Tracking Number}, and it is insured for ${Insurance Value ($)}. The parcel is marked as {Fragile} and {Temperature Sensitive}. Special instructions include: '{Special Instructions}'. There have been {Delivery Attempt Count} delivery attempts, and the last known location of the parcel was {Last Location}. The courier handling this shipment is identified by ID {Courier ID}." 



In [82]:
def populate_template(template, row):
    try:
        # Convert row to dictionary and replace NaN with 'N/A'
        row_dict = row.to_dict()
        print(row_dict)
        for key, value in row_dict.items():
            if pd.isna(value):
                row_dict[key] = 'N/A'
        paragraph = template.format(**row_dict)
        print("Paragraph:", paragraph)
    except KeyError as e:
        print(f"Missing key in template: {e}")
        paragraph = template  # Return the template as-is if keys are missing
    except Exception as e:
        print(f"Error populating template: {e}")
        paragraph = template  # Fallback to the template

    return paragraph

# Apply the populate_template function to each row in the dataframe
df['Paragraph'] = df.apply(lambda row: populate_template(template, row), axis=1)

{'Parcel ID': 'PCL398114', 'Sender Name': 'Michael Brown', 'Sender Address': '456 Maple Ave, Los Angeles, CA 90001', 'Sender Contact': '+1-555-4253', 'Receiver Name': 'Sara Davis', 'Receiver Address': '202 Birch Blvd, Phoenix, AZ 85001', 'Receiver Contact': '+1-555-8367', 'Parcel Weight (kg)': 7.75, 'Parcel Dimensions (cm)': '97x16x70', 'Shipment Date': '2024-11-17', 'Estimated Delivery Date': '2024-11-17', 'Delivery Status': 'Delivered', 'Shipping Cost ($)': 8.66, 'Shipping Method': 'Standard', 'Tracking Number': 'TRK389255020', 'Insurance Value ($)': 257.34, 'Fragile': 'Yes', 'Temperature Sensitive': 'No', 'Special Instructions': nan, 'Delivery Attempt Count': 1, 'Last Location': 'Distribution Center, Phoenix, AZ', 'Courier ID': 'CR9571', 'Paragraph': '"Parcel ID PCL398114 was shipped by Michael Brown from 456 Maple Ave, Los Angeles, CA 90001 with the contact number +1-555-4253. It is addressed to Sara Davis at 202 Birch Blvd, Phoenix, AZ 85001, who can be reached at +1-555-8367. The

In [80]:
df['Paragraph'][0]

'"Parcel ID PCL398114 was shipped by Michael Brown from 456 Maple Ave, Los Angeles, CA 90001 with the contact number +1-555-4253. It is addressed to Sara Davis at 202 Birch Blvd, Phoenix, AZ 85001, who can be reached at +1-555-8367. The parcel weighs 7.75 kg and has dimensions of 97x16x70. The shipment was sent on 2024-11-17 and is estimated to be delivered by 2024-11-17. Currently, the delivery status is \'Delivered\' and it was shipped via Standard at a cost of $8.66. The tracking number for this parcel is TRK389255020, and it is insured for $257.34. The parcel is marked as Yes and No. Special instructions include: \'N/A\'. There have been 1 delivery attempts, and the last known location of the parcel was Distribution Center, Phoenix, AZ. The courier handling this shipment is identified by ID CR9571."'

In [81]:
# Save the paragraphs to both text and PDF files
save_content_to_txt(df['Paragraph'].tolist(),
                    txt_filename='parcel_information.txt')

save_content_to_pdf(df['Paragraph'].tolist(),
                    pdf_filename='parcel_information.pdf')

All content has been saved as 'parcel_information.txt'.

All content has been saved as 'parcel_information.pdf'.

